In [3]:
import copy
import random
import sys

def generateConflicts(numCourses):
    conflicts = [[0 for i in range(numCourses)] for j in range(numCourses)]
    comboDone = []
    for i in range(numCourses):
        for j in range(numCourses):
            combo1 = (i,j)
            combo2 = (j,i)
            if combo2 in comboDone:
                conflicts[i][j] = conflicts[j][i]
                continue
            if i == j:
                conflicts[i][j] = 0
            elif random.randint(0,9) > 3:
                conflicts[i][j] = random.randint(1,20)
                
            comboDone.append(combo1)
            comboDone.append(combo2)
    
    return conflicts

def generateTable(numCourses, numHalls, timeSlots):
    coveredCombo = []
    table = []
    for i in range(numCourses):
        done = False
        while done == False:
            hall = random.randint(0,numHalls-1)
            slot = random.randint(0,timeSlots-1)
            tup = (hall,slot)
            if tup not in coveredCombo:
                coveredCombo.append(tup)
                newSlot = (i,hall,slot)
                table.append(newSlot)
                done = True
        
    return table

def isTableValid(table):
    comboDone = []
    for tup in table:
        combo = (tup[1],tup[2])
        if combo in comboDone:
            return False
        comboDone.append(combo)
    return True

def fitness(table,conflicts):
    numConflicts = 0
    for tup1 in table:
        for tup2 in table:
            if tup1[2] == tup2[2]:
                numConflicts += conflicts[tup1[0]][tup2[0]]
    
    numConflicts *= 50
    return numConflicts

def printTable(table):
    for tup in table:
        print("(C",tup[0],",H",tup[1],",T",tup[2],")", end = ",")

def crossOver(table1, table2):
    
    success = False
    tried = []
    newTable = []
    
    while success == False:
        if len(tried) == len(table1)-1:
            return None
        slider = random.randint(1,len(table1)-1)
        if slider in tried:
            continue
        newTable = table1[:slider] + table2[slider:]
        tried.append(slider)
        if isTableValid(newTable) == True:
            return newTable

def mutation(table1, numHalls, numTimeSlot):
    success = False
    tried = []
    newTable = []
    
    while success == False:
        if len(tried) == len(table1)-1:
            return None
        mutPos = random.randint(0,len(table1)-1)
        if mutPos in tried:
            continue
        newTable = copy.deepcopy(table1)
        tupPos = random.randint(1,2)
        
        tempTuple = newTable[mutPos]
        if (tupPos == 1):
            newTuple = (tempTuple[0],random.randint(0,numHalls),tempTuple[2])
            while newTuple == tempTuple:
                newTuple = (tempTuple[0],random.randint(0,numHalls),tempTuple[2])
            newTable[mutPos] = newTuple
        if (tupPos == 2):
            newTuple = (tempTuple[0],tempTuple[1], random.randint(0,numTimeSlot))
            while newTuple == tempTuple:
                newTuple = (tempTuple[0],tempTuple[1], random.randint(0,numTimeSlot))
            newTable[mutPos] = newTuple
        
        tried.append(mutPos)
        if isTableValid(newTable) == True:
            return newTable

numCourses = int(input("Enter number of courses: "))
while numCourses < 1:
    numCourses = int(input("Cannot be less than 1. Re-enter number of courses: "))

numHalls = int(input("Enter number of Halls: "))
while numHalls < 1:
    numHalls = int(input("Cannot be less than 1. Re-enter number of Halls: "))

timeSlots = int(input("Enter number of timeslots: "))
while timeSlots < 1 or timeSlots < numCourses/numHalls:
    timeSlots = int(input("Cannot be less than 1 and can't be less than number of courses x number of Halls. Re-enter number of timeslots: "))
    
conflicts = generateConflicts(numCourses)

size = int(input("Enter population size: "))
while size < 2:
    size = int(input("Cannot be less than 2 as crossover can't be done. Re-enter population size: "))

tournament = int(input("Enter the tournament size: "))
while (tournament > size or tournament < 2):
    tournament = int(input("Cannot be greater than population size or less than 2. Re-enter tournament size: "))

generations = int(input("Enter number of generations: "))
while generations < 1:
    generations = int(input("Invalid number entered. Re-enter number of generations: "))


population = []
for i in range(size):
    tableTup = (-1,-1,-1)
    while(1):
        newTable = generateTable(numCourses, numHalls, timeSlots)
        fitnessLevel = fitness(newTable,conflicts)
        tableTup = (fitnessLevel,newTable)
        if tableTup in population:
            continue
        population.append(tableTup)
        break

population.sort(key=lambda a: a[0])

bestFit = []
for i in range(tournament):
    bestFit.append(population[i])

for i in range(generations):
    parent1Tup = bestFit[0]
    if parent1Tup[0] == 0:
        print("Solution found in generation number: ", i)
        print("FITNESS: ", parent1Tup[0])
        printTable(parent1Tup[1])
        sys.exit()
    parent2Index = random.randint(1,tournament-1)
    parent2Tup = bestFit[parent2Index]
    childTable = crossOver(parent1Tup[1], parent2Tup[1])
    if childTable != None:
        childMutation = mutation(childTable, numHalls, timeSlots)
        if childMutation != None:
            
            childTup = (None,None)
            if (fitness(childMutation, conflicts) < fitness(childTable, conflicts)):
                childTup = (fitness(childTable, conflicts), childTable)
            else:
                childTup = (fitness(childMutation, conflicts), childMutation)
        else:
            childTup = (fitness(childTable, conflicts), childTable)
            
        if childTup in bestFit:
            continue
        if (childTup[0] <= bestFit[tournament-1][0]):
            bestFit[tournament-1] = childTup
    
    bestFit.sort(key=lambda a: a[0])

print("After ", generations, " generations, best solution found is: ")
print("FITNESS", bestFit[0][0])
printTable(bestFit[0][1])



Enter number of courses: 3
Enter number of Halls: 4
Enter number of timeslots: 5
Enter population size: 100
Enter the tournament size: 2
Enter number of generations: 100
Solution found in generation number:  0
FITNESS:  0
(C 0 ,H 0 ,T 4 ),(C 1 ,H 2 ,T 0 ),(C 2 ,H 3 ,T 2 ),

SystemExit: 

C:\Users\Hp OMEN\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
